In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

# Configuration parameters
DATA_PATH = "Data.xlsx" 
MODEL_PATHS = {
    'qP': 'qP_final_model.h5',
    'mu': 'mu_final_model.h5'
}
FEATURE_NAMES = ['OUR', 'CER', 'DO', 'pH', 
                'TS', 'RS', 'AN'] 
SAMPLE_INDEX = 42  # Index of samples selected for analysis
N_BACKGROUND = 100  # SHAP background sample size

# Data Preparation Functions
def prepare_data(target):
    # Load Raw Data
    df = pd.read_excel(DATA_PATH)
    X = df.iloc[:, :7].values
    y = df[target].values
    
    # Data normalization (using previously saved scaler)
    scaler_X = StandardScaler().fit(X)
    X_scaled = scaler_X.transform(X)
    
    return X_scaled, scaler_X

# SHAP Analysis Master Functions
def shap_analysis(target):
    print(f"\n=== Analysis in progress {target} model ===")
    
    # Loading models and data
    model = load_model(MODEL_PATHS[target])
    X_scaled, scaler_X = prepare_data(target)
    
    # Preparation of background and test data
    background = shap.utils.sample(X_scaled, N_BACKGROUND)
    test_sample = X_scaled[SAMPLE_INDEX:SAMPLE_INDEX+1]
    
    # Creating an Interpreter
    explainer = shap.DeepExplainer(
        model,
        background
    )
    
    # Calculating the SHAP value
    shap_values = explainer.shap_values(test_sample)
    
    # visualization and analysis
    plt.figure(figsize=(10, 6))
    shap.summary_plot(
        explainer.shap_values(background),
        features=background,
        feature_names=FEATURE_NAMES,
        show=False
    )
    plt.title(f'{target} Model Feature Importance')
    plt.tight_layout()
    plt.savefig(f'{target}_shap_summary.png')
    plt.close()
    
    # Individual sample analysis
    plt.figure()
    shap.decision_plot(
        explainer.expected_value.numpy(),
        shap_values[0], 
        test_sample[0],
        feature_names=FEATURE_NAMES,
        show=False
    )
    plt.title(f'{target} Prediction Breakdown')
    plt.tight_layout()
    plt.savefig(f'{target}_decision_plot.png')
    plt.close()
    
    # Generate dependency graphs
    for feat_idx in range(7):
        plt.figure()
        shap.dependence_plot(
            feat_idx,
            explainer.shap_values(background)[0],
            features=background,
            feature_names=FEATURE_NAMES,
            show=False
        )
        plt.title(f'{target} Feature {FEATURE_NAMES[feat_idx]} Dependency')
        plt.tight_layout()
        plt.savefig(f'{target}_dep_{FEATURE_NAMES[feat_idx]}.png')
        plt.close()
    
    # Save SHAP value data
    np.savez(f'{target}_shap_values.npz',
            shap_values=shap_values,
            sample_values=test_sample,
            feature_names=FEATURE_NAMES)
    
    print(f"{target} Analysis complete, results saved")

# Interactive Visualization Functions
def interactive_shap_analysis(target):
    # Load data
    X_scaled, _ = prepare_data(target)
    model = load_model(MODEL_PATHS[target])
    
    # Creating JS Visualizations
    explainer = shap.DeepExplainer(model, X_scaled[:N_BACKGROUND])
    shap_values = explainer.shap_values(X_scaled[:10])
    
    # Generate interactive visualizations
    shap.initjs()
    force_plot = shap.force_plot(
        explainer.expected_value.numpy(),
        shap_values[0][0], 
        X_scaled[0],
        feature_names=FEATURE_NAMES
    )
    shap_html = f"<head>{shap.getjs()}</head>{force_plot.html()}"
    
    with open(f'{target}_interactive_shap.html', 'w') as f:
        f.write(shap_html)

if __name__ == "__main__":
    # Implementation of the core analysis
    for target in ['qP', 'mu']:
        shap_analysis(target)
    
    # Generate interactive visualizations
    interactive_shap_analysis('qP')
    interactive_shap_analysis('mu')